In [1]:
!mkdir -p ~/.kaggle
from google.colab import drive
from google.colab import files
# Upload your kaggle.json API key
uploaded = files.upload()

drive.mount('/content/drive')

Saving kaggle.json to kaggle.json
Mounted at /content/drive


In [16]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout, Concatenate
import pandas as pd
import numpy as np

In [17]:
import os
import shutil
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/Machine-learning'

In [18]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Machine-learning/hmnist_28_28_RGB.csv')

# Extract features and labels
x = data.drop(columns=['label']).values
y = data['label'].values

In [19]:
classes = {4: ('nv', ' melanocytic nevi'), 6: ('mel', 'melanoma'), 2 :('bkl', 'benign keratosis-like lesions'), 1:('bcc' , ' basal cell carcinoma'), 5: ('vasc', ' pyogenic granulomas and hemorrhage'), 0: ('akiec', 'Actinic keratoses and intraepithelial carcinomae'),  3: ('df', 'dermatofibroma')}

In [20]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler()
x,y  = oversample.fit_resample(x,y)
x = np.array(x).reshape(-1,28,28,3)
print('Shape of X :',x.shape)
from sklearn.model_selection import train_test_split
x = (x-np.mean(x))/np.std(x)
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size=0.2, random_state=1)

Shape of X : (46935, 28, 28, 3)


In [25]:
original_image_height = 28
original_image_width = 28
original_image_channels = 3

# Reshape the images to their original shape
x_original_shape = x_flattened.reshape(-1, original_image_height, original_image_width, original_image_channels)

# Split the data into training, validation, and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(x_original_shape, y, test_size=0.4, random_state=1)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=1)


NameError: ignored

In [21]:
def inception_module(x, filters):
    conv1x1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)
    conv3x3_reduce = Conv2D(filters[1], (1, 1), padding='same', activation='relu')(x)
    conv3x3 = Conv2D(filters[2], (3, 3), padding='same', activation='relu')(conv3x3_reduce)
    conv5x5_reduce = Conv2D(filters[3], (1, 1), padding='same', activation='relu')(x)
    conv5x5 = Conv2D(filters[4], (5, 5), padding='same', activation='relu')(conv5x5_reduce)
    maxpool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    maxpool_proj = Conv2D(filters[5], (1, 1), padding='same', activation='relu')(maxpool)
    inception_output = Concatenate(axis=-1)([conv1x1, conv3x3, conv5x5, maxpool_proj])
    return inception_output


In [22]:
def googlenet(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # Initial Convolution Layer
    conv1 = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_layer)
    maxpool1 = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(conv1)

    # Inception Modules
    inception1 = inception_module(maxpool1, [64, 96, 128, 16, 32, 32])
    inception2 = inception_module(inception1, [128, 128, 192, 32, 96, 64])

    # Add more Inception modules as needed

    # Fully Connected Layers
    avgpool = AveragePooling2D((7, 7), strides=(1, 1), padding='valid')(inception2)
    flatten = Flatten()(avgpool)
    dropout = Dropout(0.4)(flatten)
    output_layer = Dense(num_classes, activation='softmax')(dropout)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Create the model
input_shape = (32, 32, 3)  # Adjust input shape as needed
num_classes = 7  # Number of classes in HAM10000 dataset
model = googlenet(input_shape, num_classes)
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_13 (Conv2D)             (None, 16, 16, 64)   9472        ['input_2[0][0]']                
                                                                                                  
 max_pooling2d_3 (MaxPooling2D)  (None, 8, 8, 64)    0           ['conv2d_13[0][0]']              
                                                                                                  
 conv2d_15 (Conv2D)             (None, 8, 8, 96)     6240        ['max_pooling2d_3[0][0]']        
                                                                                            

In [23]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train,
                    Y_train,
                    validation_data=(X_val, Y_val),
                    batch_size=128,
                    epochs=30,
                    callbacks=[checkpoint_callback])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
print("Test Accuracy:", accuracy)

# Calculate individual class accuracy
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
cm = confusion_matrix(Y_test, y_pred_classes)
class_accuracy = cm.diagonal() / cm.sum(axis=1)
for i, acc in enumerate(class_accuracy):
    print(f"Class {i}: Accuracy = {acc:.4f}")


NameError: ignored